In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [2]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|distributiongloba...|      false|
| tabd_db|globalinternetusa...|      false|
| tabd_db|globalmobilepenet...|      false|
| tabd_db|globalsocialnetworks|      false|
| tabd_db| internetactivities1|      false|
| tabd_db| internetactivities2|      false|
| tabd_db|       internetusage|      false|
| tabd_db|internetusagefreq...|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|p_dailyinternetus...|      false|
| tabd_db|p_enterprisesocia...|      false|
| tabd_db|p_globalsocialmed...|      false|
| tabd_db|p_globalsocialnet...|      false|
| tabd_db|p_individualsacti...|      false|
| tabd_db|     p_internetusage|      false|
| tabd_db|parentawarenessof...|      false|
| tabd_db|socialmediausagef...|      false|
| tabd_db|socialmediausebyp...|      false|
| tabd_db|socialmediausebyt...| 

In [3]:
spark.sql(
    """
    DROP TABLE IF EXISTS tabd_db.AdultInternetUsagePenetration
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE tabd_db.AdultInternetUsagePenetration (
        year INT,
        region VARCHAR(45),
        gender VARCHAR(45),
        usage_rate DOUBLE,
        penetration_percentage INT,
        internet_users DOUBLE,
        frequency_of_access VARCHAR(45),
        individuals VARCHAR(45),
        frequency_of_access_percentage INT

    )
    STORED AS PARQUET
    LOCATION 'hdfs://hdfs-nn:9000/warehouse/tabd.db/AdultInternetUsagePenetration/'
    """
)

        

# tblproperties('skip.header.line.count'='1')
# can be used for csvs with header
# but spark sql cannot understand that at the moment, while reading the data using sql queries
# so just when creating hive tables backed up by csvs, avoid headers
# in the project we wont use hive text tables, so all good.

DataFrame[]

In [4]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|adultinternetusag...|      false|
| tabd_db|distributiongloba...|      false|
| tabd_db|globalinternetusa...|      false|
| tabd_db|globalmobilepenet...|      false|
| tabd_db|globalsocialnetworks|      false|
| tabd_db| internetactivities1|      false|
| tabd_db| internetactivities2|      false|
| tabd_db|       internetusage|      false|
| tabd_db|internetusagefreq...|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|p_dailyinternetus...|      false|
| tabd_db|p_enterprisesocia...|      false|
| tabd_db|p_globalsocialmed...|      false|
| tabd_db|p_globalsocialnet...|      false|
| tabd_db|p_individualsacti...|      false|
| tabd_db|     p_internetusage|      false|
| tabd_db|parentawarenessof...|      false|
| tabd_db|socialmediausagef...|      false|
| tabd_db|socialmediausebyp...| 

In [5]:
# Let's look into HDFS

In [6]:
spark.sql(
    """
    SELECT *
    FROM tabd_db.AdultInternetUsagePenetration
    """
).show()

+----+------+------+----------+----------------------+--------------+-------------------+-----------+------------------------------+
|year|region|gender|usage_rate|penetration_percentage|internet_users|frequency_of_access|individuals|frequency_of_access_percentage|
+----+------+------+----------+----------------------+--------------+-------------------+-----------+------------------------------+
|2000|  null|  male|       0.0|                    54|           0.0|               null|       null|                             0|
|2000|  null|female|       0.0|                    50|           0.0|               null|       null|                             0|
|2005|  null|  male|       0.0|                    69|           0.0|               null|       null|                             0|
|2005|  null|female|       0.0|                    67|           0.0|               null|       null|                             0|
|2009|  null|  male|       0.0|                    77|           0.0|

In [7]:
spark.sql(
    """
    DESCRIBE FORMATTED tabd_db.AdultInternetUsagePenetration
    """
).toPandas()

,col_name,data_type,comment
0,year,int,None
1,region,varchar(45),None
2,gender,varchar(45),None
3,usage_rate,double,None
4,penetration_percentage,int,None
5,internet_users,double,None
6,frequency_of_access,varchar(45),None
7,individuals,varchar(45),None
8,frequency_of_access_percentage,int,None
9,,,


In [8]:
# Let's put the files into HDFS

In [9]:
spark.sql(
    """
    SELECT *
    FROM tabd_db.AdultInternetUsagePenetration
    """
).toPandas()

,year,region,gender,usage_rate,penetration_percentage,internet_users,frequency_of_access,individuals,frequency_of_access_percentage
0,2000,null,male,0.0,54,0.0,null,null,0
1,2000,null,female,0.0,50,0.0,null,null,0
2,2005,null,male,0.0,69,0.0,null,null,0
3,2005,null,female,0.0,67,0.0,null,null,0
4,2009,null,male,0.0,77,0.0,null,null,0
5,2009,null,female,0.0,75,0.0,null,null,0
6,2010,null,male,0.0,77,0.0,null,null,0
7,2010,null,female,0.0,76,0.0,null,null,0
8,2013,null,male,0.0,84,0.0,null,null,0
9,2013,null,female,0.0,84,0.0,null,null,0


In [10]:


spark.sql(
    """
    SELECT *
    FROM tabd_db.AdultInternetUsagePenetration
    """
).show()

+----+------+------+----------+----------------------+--------------+-------------------+-----------+------------------------------+
|year|region|gender|usage_rate|penetration_percentage|internet_users|frequency_of_access|individuals|frequency_of_access_percentage|
+----+------+------+----------+----------------------+--------------+-------------------+-----------+------------------------------+
|2000|  null|  male|       0.0|                    54|           0.0|               null|       null|                             0|
|2000|  null|female|       0.0|                    50|           0.0|               null|       null|                             0|
|2005|  null|  male|       0.0|                    69|           0.0|               null|       null|                             0|
|2005|  null|female|       0.0|                    67|           0.0|               null|       null|                             0|
|2009|  null|  male|       0.0|                    77|           0.0|

In [11]:
spark.stop()